# ASE & CP2K

Set some environment variables for the notebook. For doing this on a HPC resource, these would be set as export commands in the slurm script. Also, the ASE_CP2K_COMMAND would change to "srun cp2k_shell.psmp"

In [3]:
%set_env SHELL=/bin/bash
%set_env OMP_NUM_THREADS=8
%set_env VECLIB_MAXIMUM_THREADS=1
%set_env ASE_CP2K_COMMAND=cp2k_shell.ssmp

env: SHELL=/bin/bash
env: OMP_NUM_THREADS=8
env: VECLIB_MAXIMUM_THREADS=1
env: ASE_CP2K_COMMAND=cp2k_shell.ssmp


## Modules

In [5]:
# General
import os
import sys
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt

# For building things
from ase import Atom, Atoms
from ase.io import read, write
from ase.io.trajectory import Trajectory
from ase.build import molecule
from ase.visualize import view

# Unit Conversions and Fixing Atoms
from ase.units import Bohr,Rydberg,kJ,kB,fs,Hartree,mol,kcal
from ase.constraints import FixedPlane, FixedLine, FixAtoms

# ASE Calculators
from plumed import Plumed
from ase.calculators.cp2k import CP2K
from ase.calculators.lj import LennardJones
from ase.calculators.plumed import Plumed
from ase.calculators.idealgas import IdealGas

# Geometry Optimizations and Normal Mode Analysis
from ase.optimize import LBFGS, FIRE
from ase.vibrations import Vibrations
from ase.thermochemistry import IdealGasThermo

# EOS fitting for Unit Cells
from ase.eos import EquationOfState

# Molecular Dynamics
from ase.md.velocitydistribution import MaxwellBoltzmannDistribution
from ase.md.verlet import VelocityVerlet
from ase.md.langevin import Langevin
from ase.md.npt import NPT

cwd = os.getcwd()

In [6]:
def myCP2KCalculator(tag, functional, ecut):
    """Creates a CP2K calculator object with Settings for Production Runs
    tag -> label for cp2k output
    functional -> Either PBE+D3, BEEFVDW, rVV10, or LDA+FermiDirac
    ecut -> PW Kinetic Energy Cutoff (Rydberg)
    """
    
    if functional == "PBE+D3":
        xc = "PBE"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &VDW_POTENTIAL
        POTENTIAL_TYPE PAIR_POTENTIAL
        &PAIR_POTENTIAL
          R_CUTOFF 15.0
          TYPE DFTD3
          CALCULATE_C9_TERM .FALSE.
          REFERENCE_FUNCTIONAL PBE
          PARAMETER_FILE_NAME dftd3.dat
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI OFF
          OUTPUT_TEXT .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "BEEFVDW":
        xc = "GGA_XC_BEEFVDW"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
      &VDW_POTENTIAL
          &NON_LOCAL
              TYPE LMKLL
              KERNEL_FILE_NAME vdW_kernel_table.dat
          &END NON_LOCAL
      &END VDW_POTENTIAL
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI ON
          OUTPUT_TEXT .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "rVV10":
        xc = None
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    SURFACE_DIPOLE_CORRECTION .FALSE.
    SURF_DIP_DIR Z
    &SCF
      &OT .TRUE.
        MINIMIZER DIIS
        PRECONDITIONER FULL_SINGLE_INVERSE
      &END OT
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
       &XC_FUNCTIONAL
         &GGA_X_RPW86
         &END GGA_X_RPW86
         &GGA_C_PBE
         &END GGA_C_PBE
       &END XC_FUNCTIONAL
       &vdW_POTENTIAL
          DISPERSION_FUNCTIONAL NON_LOCAL
          &NON_LOCAL
            TYPE RVV10
            PARAMETERS 6.3 0.0093
            VERBOSE_OUTPUT
            KERNEL_FILE_NAME rVV10_kernel_table.dat
            CUTOFF 40
          &END NON_LOCAL
       &END vdW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI ON
          OUTPUT_TEXT .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    elif functional == "LDA+FermiDirac":
        xc = "LDA"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    &KPOINTS
        SCHEME MONKHORST-PACK 6 6 6
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI OFF
          OUTPUT_EMP .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
        
    elif functional == "PBED3+FermiDirac":
        xc = "PBE"
        inp = '''
&GLOBAL
  WALLTIME 47:58:00
&END GLOBAL
&FORCE_EVAL
  &DFT
    &KPOINTS
        SCHEME MONKHORST-PACK 6 6 6
    &END KPOINTS
    &SCF
      ADDED_MOS 10
      &SMEAR ON
          METHOD FERMI_DIRAC
          ELECTRONIC_TEMPERATURE [K] 300
      &END SMEAR
      &MIXING .TRUE.
          METHOD BROYDEN_MIXING
      &END MIXING
      &OUTER_SCF .TRUE.
        MAX_SCF 50
      &END OUTER_SCF
    &END SCF
    &XC
      &VDW_POTENTIAL
        POTENTIAL_TYPE PAIR_POTENTIAL
        &PAIR_POTENTIAL
          R_CUTOFF 15.0
          TYPE DFTD3
          CALCULATE_C9_TERM .FALSE.
          REFERENCE_FUNCTIONAL PBE
          PARAMETER_FILE_NAME dftd3.dat
        &END PAIR_POTENTIAL
      &END VDW_POTENTIAL
      &XC_GRID
        XC_DERIV NN10_SMOOTH
        XC_SMOOTH_RHO NN10
      &END
    &END XC
    &PRINT
      &E_DENSITY_BQB OFF
      &END E_DENSITY_BQB
      &VORONOI OFF
          OUTPUT_EMP .TRUE.
      &END VORONOI
    &END PRINT
  &END DFT
&END FORCE_EVAL
'''
    else:
        xc = "LDA"
        inp = ''''''
        
    calc = CP2K(
        auto_write=False,
        basis_set="DZVP-MOLOPT-SR-GTH",
        basis_set_file="BASIS_MOLOPT",
        charge=0,
        cutoff = ecut*Rydberg,
        debug = False,
        force_eval_method = "Quickstep",
        xc = xc,
        inp = inp,
        max_scf = 50,
        poisson_solver ="auto",
        potential_file = "POTENTIAL",
        pseudo_potential = "GTH-PBE",
        stress_tensor = True,
        print_level = "LOW",
        label = tag,
    )
    return calc

## Molecule Optimization

In [7]:
def MyFIREOptimization(atoms, dirname):

    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    else:
        pass
    os.chdir(dirname)
    
    atoms.calc = myCP2KCalculator(dirname, "PBE+D3", 400.0)
    
    minimizer = FIRE(atoms, trajectory="fire.traj", logfile="fire.log")
    minimizer.run(fmax=0.03)
    potentialenergy = atoms.get_potential_energy()
    
    os.chdir(cwd)

def MyLBFGSOptimization(atoms, dirname):

    if not os.path.isdir(dirname):
        os.mkdir(dirname)
    else:
        pass
    os.chdir(dirname)
    
    atoms.calc = myCP2KCalculator(dirname, "PBE+D3", 400.0)
    
    minimizer = LBFGS(atoms, trajectory="lbfgs.traj", logfile="lbfgs.log")
    minimizer.run(fmax=0.03)
    potentialenergy = atoms.get_potential_energy()
    
    os.chdir(cwd)

    

## Furfural and Derivatives

In [8]:
fal = read("Resources/xyz/furfural/FAL.xyz")
fal.center(vacuum=6.0)
fal.set_pbc(True)
view(fal, viewer = "ngl")

In [9]:
MyFIREOptimization(fal, "simulations-molecule-fal-minimization")

In [10]:
view(fal, viewer = "ngl")

In [13]:
write("Resources/xyz/furfural/FAL.xyz",fal)

In [15]:
fol = read("Resources/xyz/furfural/FOL.xyz")
fol.center(vacuum=6.0)
fol.set_pbc(True)
view(fol, viewer = "ngl")

In [16]:
MyFIREOptimization(fol, "simulations-molecule-fol-minimization")

In [17]:
view(fol, viewer = "ngl")

In [18]:
write("Resources/xyz/furfural/FOL.xyz",fol)

In [19]:
mf = read("Resources/xyz/furfural/MF.xyz")
mf.center(vacuum=6.0)
mf.set_pbc(True)
view(mf, viewer = "ngl")

In [20]:
MyFIREOptimization(mf, "simulations-molecule-mf-minimization")

In [21]:
view(mf, viewer = "ngl")

In [22]:
write("Resources/xyz/furfural/MF.xyz",mf)

In [23]:
thfol = read("Resources/xyz/furfural/THFOL.xyz")
thfol.center(vacuum=6.0)
thfol.set_pbc(True)
view(thfol, viewer = "ngl")

In [24]:
MyFIREOptimization(thfol, "simulations-molecule-thfol-minimization")

In [25]:
view(thfol, viewer = "ngl")

In [26]:
write("Resources/xyz/furfural/THFOL.xyz",thfol)

## GG Lignin Linkages

In [17]:
bo4 = read("Resources/xyz/lignin/GG/BO4.xyz")
bo4.center(vacuum=6.0)
bo4.set_pbc(True)
view(bo4, viewer = "ngl")

In [18]:
MyFIREOptimization(bo4, "simulations-molecule-bo4-minimization")

In [19]:
view(bo4, viewer = "ngl")

In [20]:
write("Resources/xyz/lignin/GG/BO4.xyz",bo4)

/home/woodrowwilson/Programs/ase/dev/ase/io/extxyz.py:1001: UserWarning: write_xyz() overwriting array "forces" present in atoms.arrays with stored results from calculator
  warnings.warn('write_xyz() overwriting array "{0}" present '


In [8]:
pe = read("Resources/xyz/lignin/PE/PE.xyz")
pe.center(vacuum=6.0)
pe.set_pbc(True)
view(pe, viewer = "ngl")

In [9]:
MyFIREOptimization(pe, "simulations-molecule-pe-minimization")

In [10]:
view(pe, viewer = "ngl")

In [11]:
write("Resources/xyz/lignin/PE/PE.xyz",pe)

In [14]:
hh = read("Resources/xyz/lignin/HH/HH.xyz")
hh.center(vacuum=6.0)
hh.set_pbc(True)
view(hh, viewer = "ngl")

In [15]:
MyFIREOptimization(hh, "simulations-molecule-hh-minimization")

In [16]:
write("Resources/xyz/lignin/HH/HH.xyz",hh)

In [23]:
pd = read("Resources/xyz/lignin/PD/PD.xyz")
pd.center(vacuum=6.0)
pd.set_pbc(True)
view(pd, viewer = "ngl")

In [24]:
MyFIREOptimization(pd, "simulations-molecule-pd-minimization")

In [30]:
import glob
os.chdir(cwd)
solvents = glob.glob("Resources/xyz/solvents/*.xyz")
for solvent in solvents:
    print(solvent)
    atoms = read(solvent)
    atoms.center(vacuum=6.0)
    atoms.set_pbc(True)
    MyFIREOptimization(atoms, "simulations-molecule-minimization")
    write(solvent, atoms)


Resources/xyz/solvents/C10H12.xyz


/home/woodrowwilson/Programs/ase/dev/ase/io/extxyz.py:1001: UserWarning: write_xyz() overwriting array "forces" present in atoms.arrays with stored results from calculator
  warnings.warn('write_xyz() overwriting array "{0}" present '


Resources/xyz/solvents/NH3.xyz
Resources/xyz/solvents/CH3OH.xyz
Resources/xyz/solvents/C5H5N.xyz
Resources/xyz/solvents/H2O.xyz
Resources/xyz/solvents/CH2O2.xyz
